In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from service.utils.spark import get_spark_session
# spark = SparkSession.builder.getOrCreate()
spark = get_spark_session(dev=True)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/23 06:21:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
test_namespace = 'gold' 
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {test_namespace}").show()

++
||
++
++



In [3]:
payment_df = spark.read.csv("s3a://warehousedev/bronze/tsv/payment.tsv", header=True, sep='\t')

25/09/23 06:21:57 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
order_customer_id = payment_df.select('order_id', 'customer_id').distinct()

In [5]:
order_status_df = spark.read.csv("s3a://warehousedev/bronze/tsv/order_status.tsv", header=True, sep='\t')
order_status_df.show(truncate=False)


+--------------------------------+--------+--------------------------+
|order_id                        |status  |timestamp                 |
+--------------------------------+--------+--------------------------+
|2e7a8482f6fb09756ca50c10d7bfc047|purchase|2016-09-04 21:15:19.000000|
|e5fa5a7210941f7d56d0208e4e071d35|purchase|2016-09-05 00:15:34.000000|
|809a282bbd5dbcabb6f2f724fca862ec|purchase|2016-09-13 15:24:19.000000|
|bfbd0f9bdef84302105ad712db648a6c|purchase|2016-09-15 12:16:38.000000|
|bfbd0f9bdef84302105ad712db648a6c|approved|2016-09-15 12:16:38.000000|
|71303d7e93b399f5bcd537d124c0bcfa|purchase|2016-10-02 22:07:52.000000|
|3b697a20d9e427646d92567910af6d57|purchase|2016-10-03 09:44:50.000000|
|be5bc2f0da14d8071e2d45451ad119d9|purchase|2016-10-03 16:56:50.000000|
|65d1e226dfaeb8cdc42f665422522d14|purchase|2016-10-03 21:01:41.000000|
|a41c8759fbe7aab36ea07e038b2d4465|purchase|2016-10-03 21:13:36.000000|
|d207cc272675637bfed0062edffd0818|purchase|2016-10-03 22:06:03.000000|
|cd3b8

In [6]:
# format_string = 'yyyy-MM-dd HH:mm:ss.SSSSSS'
format_string = 'yyyy-MM-dd HH:mm:ss'
order_status_df = order_status_df.withColumn('timestamp_clean', F.substring(F.col('timestamp'), 1, 19)) \
                                .withColumn('timestamp', F.to_timestamp('timestamp_clean', 'yyyy-MM-dd HH:mm:ss')) \
                                .drop('timestamp_clean')
# order_status_df = order_status_df.withColumn('timestamp', F.to_timestamp('timestamp', format_string))
order_status_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [7]:
order_status_df.show()

+--------------------+--------+-------------------+
|            order_id|  status|          timestamp|
+--------------------+--------+-------------------+
|2e7a8482f6fb09756...|purchase|2016-09-04 21:15:19|
|e5fa5a7210941f7d5...|purchase|2016-09-05 00:15:34|
|809a282bbd5dbcabb...|purchase|2016-09-13 15:24:19|
|bfbd0f9bdef843021...|purchase|2016-09-15 12:16:38|
|bfbd0f9bdef843021...|approved|2016-09-15 12:16:38|
|71303d7e93b399f5b...|purchase|2016-10-02 22:07:52|
|3b697a20d9e427646...|purchase|2016-10-03 09:44:50|
|be5bc2f0da14d8071...|purchase|2016-10-03 16:56:50|
|65d1e226dfaeb8cdc...|purchase|2016-10-03 21:01:41|
|a41c8759fbe7aab36...|purchase|2016-10-03 21:13:36|
|d207cc272675637bf...|purchase|2016-10-03 22:06:03|
|cd3b8574c82b42fc8...|purchase|2016-10-03 22:31:31|
|ae8a60e4b03c5a4ba...|purchase|2016-10-03 22:44:10|
|ef1b29b591d31d57c...|purchase|2016-10-03 22:51:30|
|0a0837a5eee9e7a9c...|purchase|2016-10-04 09:06:10|
|1ff217aa612f6cd7c...|purchase|2016-10-04 09:16:33|
|1ff217aa612

In [8]:
distinct_status = order_status_df.select('status').distinct().collect()
pivot_values = [row.status for row in distinct_status]
pivot_values

['shipped',
 'canceled',
 'approved',
 'invoiced',
 'purchase',
 'delivered_carrier',
 'delivered_customer',
 'unavailable',
 'processing']

In [9]:
pivoted_order_status_df = order_status_df.groupBy('order_id') \
    .pivot('status', pivot_values) \
    .agg(F.first('timestamp'))

# pivoted_order_status_df.show(n=5)

In [10]:
pivoted_order_status_df.show(n=5)

+--------------------+-------+--------+-------------------+--------+-------------------+-------------------+-------------------+-------------------+----------+
|            order_id|shipped|canceled|           approved|invoiced|           purchase|  delivered_carrier| delivered_customer|        unavailable|processing|
+--------------------+-------+--------+-------------------+--------+-------------------+-------------------+-------------------+-------------------+----------+
|a9a93c428c6103f21...|   NULL|    NULL|2017-01-17 10:55:13|    NULL|2017-01-14 17:57:50|2017-01-20 17:58:58|2017-01-30 18:37:44|               NULL|      NULL|
|56ef80c564f6fd57c...|   NULL|    NULL|2017-01-16 14:35:17|    NULL|2017-01-16 14:24:22|2017-01-16 15:21:56|2017-01-23 08:56:05|               NULL|      NULL|
|f9427374480e37251...|   NULL|    NULL|2017-01-18 02:10:16|    NULL|2017-01-17 14:57:45|2017-01-19 09:12:51|2017-01-24 15:14:01|               NULL|      NULL|
|d6d7c431275f0029d...|   NULL|    NULL|2

In [11]:
pivoted_order_customer_df = pivoted_order_status_df.join(order_customer_id, on='order_id', how='inner')
pivoted_order_customer_df.show()

+--------------------+-------+--------+-------------------+--------+-------------------+-------------------+-------------------+-------------------+----------+--------------------+
|            order_id|shipped|canceled|           approved|invoiced|           purchase|  delivered_carrier| delivered_customer|        unavailable|processing|         customer_id|
+--------------------+-------+--------+-------------------+--------+-------------------+-------------------+-------------------+-------------------+----------+--------------------+
|a9a93c428c6103f21...|   NULL|    NULL|2017-01-17 10:55:13|    NULL|2017-01-14 17:57:50|2017-01-20 17:58:58|2017-01-30 18:37:44|               NULL|      NULL|ef026b181b8edf96c...|
|56ef80c564f6fd57c...|   NULL|    NULL|2017-01-16 14:35:17|    NULL|2017-01-16 14:24:22|2017-01-16 15:21:56|2017-01-23 08:56:05|               NULL|      NULL|6bd4f06d5599d085d...|
|f9427374480e37251...|   NULL|    NULL|2017-01-18 02:10:16|    NULL|2017-01-17 14:57:45|2017-01

In [12]:
from functools import reduce

# stauts가 하나라도 null인 row 확인. 격리 필요
conditions = [F.isnull(F.col(c)) for c in pivoted_order_customer_df.columns]
final_condition = reduce(lambda a, b: a | b, conditions)
rows_with_any_na = pivoted_order_customer_df.filter(final_condition)
rows_with_any_na.count()

99440

In [13]:
complete_order_timestamp_df = pivoted_order_customer_df.select('order_id', 'customer_id', 'purchase', 'approved', 'delivered_carrier', 'delivered_customer').dropna()
complete_order_timestamp_df.count()

96467

In [14]:
complete_order_timestamp_df.show(n=5)

+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|            order_id|         customer_id|           purchase|           approved|  delivered_carrier| delivered_customer|
+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|a9a93c428c6103f21...|ef026b181b8edf96c...|2017-01-14 17:57:50|2017-01-17 10:55:13|2017-01-20 17:58:58|2017-01-30 18:37:44|
|56ef80c564f6fd57c...|6bd4f06d5599d085d...|2017-01-16 14:24:22|2017-01-16 14:35:17|2017-01-16 15:21:56|2017-01-23 08:56:05|
|f9427374480e37251...|71f9d647cd3876fd5...|2017-01-17 14:57:45|2017-01-18 02:10:16|2017-01-19 09:12:51|2017-01-24 15:14:01|
|d6d7c431275f0029d...|3855608848a57e1e3...|2017-01-19 14:28:48|2017-01-19 14:41:56|2017-01-24 10:05:23|2017-01-31 12:26:08|
|0957ed870116e596b...|903fccb1294b588a6...|2017-01-29 22:14:49|2017-01-29 22:33:34|2017-01-30 08:27:47|2017-02-08 17:14:55|
+-------

In [15]:
complete_order_timestamp_df.writeTo(f"{test_namespace}.complete_order_timestamp").createOrReplace()

In [16]:
complete_order_timestamp_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- purchase: timestamp (nullable = true)
 |-- approved: timestamp (nullable = true)
 |-- delivered_carrier: timestamp (nullable = true)
 |-- delivered_customer: timestamp (nullable = true)



In [17]:
# complete_order_timestamp_df.show(n=3)

In [18]:
# spark.sql("describe table gold.complete_order_timestamp").show()

In [20]:
spark.stop()